# 大模型推理配置自动推荐

千帆平台上的模型提供了大量参数可供用户调整，而这些参数设置会直接影响到模型表现，并且在不同场景下，最优的配置也不尽相同。

针对这个问题，SDK 提供了推理配置自动推荐的功能，只需要提供目标场景的数据集及评估方式，设定搜索空间，SDK 就可以根据以上信息推荐出参数的配置。

In [ ]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your access key"
os.environ["QIANFAN_SECRET_KEY"] = "your secret key"
# 由于后续在调优配置的过程中会并发请求模型，建议限制 QPS 和重试次数，避免调用失败
os.environ["QIANFAN_QPS_LIMIT"] = "3"
os.environ["QIANFAN_LLM_API_RETRY_COUNT"] = "5"

## 准备工作

具体而言，在获取推荐配置前，需要先准备：

- 数据集：根据目标场景准备一定量的数据
- 评估方式：根据目标场景，选择待优化的指标，并提供评估函数

数据集使用的是千帆 SDK 中提供的 `Dataset` 模块，可以直接加载本地的数据集文件，也可以使用平台上预置的或者自行上传的数据集，具体加载方式参考 [文档](https://github.com/baidubce/bce-qianfan-sdk/blob/main/docs/dataset.md)。

In [2]:
from qianfan.dataset import Dataset

dataset = Dataset.load(
    data_file="./example.jsonl",
    organize_data_as_group=False,
    input_columns=["prompt"],
    reference_column="response",
)

[INFO] [03-28 19:51:40] dataset.py:389 [t:139919663059200]: no data source was provided, construct
[INFO] [03-28 19:51:40] dataset.py:257 [t:139919663059200]: construct a file data source from path: ./example.jsonl, with args: {'input_columns': ['prompt'], 'reference_column': 'response'}
[INFO] [03-28 19:51:40] file.py:276 [t:139919663059200]: use format type FormatType.Jsonl
[INFO] [03-28 19:51:40] utils.py:258 [t:139919663059200]: start to get memory_map from /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/autotuner/example.arrow
[INFO] [03-28 19:51:40] utils.py:233 [t:139919663059200]: has got a memory-mapped table


评估采用的 SDK 提供的 Evaluator 模块，基于 Evaluator 实现 evaluate 方法即可。如下实现了一个利用大模型评分实现评估的 Evaluator，关于如何实现 Evaluator 可以参考 [该cookbook](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/evaluation/local_eval_with_qianfan.ipynb)。

In [3]:
from qianfan.evaluation.evaluator import LocalEvaluator
from qianfan import ChatCompletion
from qianfan.common.prompt.prompt import Prompt
from qianfan.utils.pydantic import Field

from typing import Optional, Union, Any, Dict, List
import re
import json

class LocalJudgeEvaluator(LocalEvaluator):
    model: Optional[ChatCompletion] = Field(default=None, description="model object")
    eval_prompt: Prompt = Field(
        default=Prompt(
            template="""你需要扮演一个裁判的角色，对一段角色扮演的对话内容进行打分，你需要考虑这段文本中的角色沉浸度和对话文本的通畅程度。你可以根据以下规则来进行打分，你可以阐述你对打分标准的理解后再给出分数：
                "4":完全可以扮演提问中的角色进行对话，回答完全符合角色口吻和身份，文本流畅语句通顺
                "3":扮演了提问中正确的角色，回答完全符合角色口吻和身份，但文本不流畅或字数不满足要求
                "2":扮演了提问中正确的角色，但是部分语句不符合角色口吻和身份，文本流畅语句通顺
                "1":能够以角色的口吻和身份进行一部分对话，和角色设定有一定偏差，回答内容不流畅，或不满足文本字数要求
                "0":扮演了错误的角色，没有扮演正确的角色，角色设定和提问设定差异极大，完全不满意
                你的回答需要以json代码格式输出：
                ```json
                {"modelA": {"justification": "此处阐述对打分标准的理解", "score": "此处填写打分结果"}}
                ```

                现在你可以开始回答了：
                问题：{{input}}
                ---
                modelA回答：{{output}}
                ---""",
            identifier="{{}}",
        ),
        description="evaluation prompt",
    )

    class Config:
        arbitrary_types_allowed = True

    def evaluate(
        self, input: Union[str, List[Dict[str, Any]]], reference: str, output: str
    ) -> Dict[str, Any]:
        score = 0
        try:
            # 渲染评估用的 prompt，传入输入、模型输出和参考答案
            p, _ = self.eval_prompt.render(
                **{
                    "input": "\n".join([i["content"] for i in input[1:]]),
                    "output": output,
                    "expect": reference,
                }
            )
            # 请求模型进行评估
            r = self.model.do(messages=[{"role": "user", "content": p}])
            content = r["result"]
            # 提取出 json 格式的评估结果
            regex = re.compile("\`\`\`json(.*)\`\`\`", re.MULTILINE | re.DOTALL)

            u = regex.findall(content)

            if len(u) == 0:
                score = 0
            else:
                score = float(json.loads(u[0])["modelA"]["score"])
        except Exception as e:
            score = 0
        # 返回评估结果，这里字段需与后续推荐配置时设定的评估字段一致
        return {"score": score}

## 获取推荐配置

为了获取推荐配置，还需要设置一个超参搜索空间，千帆 SDK 提供了如下表示搜索空间的类：

- `Uniform`：表示一个均匀分布的搜索空间，包含两个参数 `low` 和 `high`，分别表示下界和上界。
- `Categorical`：表示一个离散的搜索空间，包含一个参数 `choices`，表示一组候选值。

In [4]:
from qianfan.autotuner.space import Uniform, Categorical

search_space = {
    "temperature": Uniform(0.01, 0.99),  # 设定temperature的范围
    "model": Categorical(["ERNIE-Speed", "ERNIE-Bot-turbo"]),  # 设定model的取值范围
    # 更多其他参数也可以按同样方式设定
}

之后就可以执行推荐

In [6]:
import qianfan.autotuner

context = await qianfan.autotuner.run(
    search_space=search_space,
    dataset=dataset,
    evaluator=LocalJudgeEvaluator(
        model=ChatCompletion(model="ERNIE-Bot-4")
    ),
    # 以下均为可选参数
    suggestor="random",  # 搜索算法，目前仅支持 "random"，更多算法敬请期待
    cost_budget=1,       # 设定整个流程的预算，达到预算则终止流程，单位为 “元”
    metrics="score",     # 设定评估指标字段，与 Evaluator 输出对应
    mode="max",          # 设定评估指标最大化还是最小化
    repeat=3,            # 重复推理次数，用于减少大模型输出随机性对结果准确性的干扰
    max_turn=10,         # 设定最大尝试次数
    max_time=3600,       # 设定最大尝试时间，单位为秒
    log_dir= "./log",    # 日志目录
)

[INFO] [03-28 19:51:48] launcher.py:98 [t:139919663059200]: turn 0 started...
[INFO] [03-28 19:51:48] launcher.py:99 [t:139919663059200]: suggested config list: [{'temperature': 0.09690927085239126, 'model': 'ERNIE-Speed'}]
[INFO] [03-28 19:51:48] dataset.py:883 [t:139919663059200]: list local dataset data by None
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]: async requesting llm api endpoint: /chat/ernie_speed
[INFO] [03-28 19:51:48] openapi_requestor.py:377 [t:139919663059200]:

返回的结果是一个 `Context` 对象，其中包含了整个搜索过程的所有上下文信息，例如可以通过如下方式获得搜索的最佳参数

In [11]:
context.best

{'temperature': 0.22907952895473882, 'model': 'ERNIE-Speed'}

这个参数直接用于推理

In [ ]:
chat = qianfan.ChatCompletion().do(messages=[{
    "role": "user",
    "messages": "请扮演一个角色，然后说一句话"
}], **context.best)

print(chat['result'])

context 中也包含了整个过程中尝试的记录，可以获取某一轮某一组配置的评估结果等信息

In [8]:
context.history[0][0].metrics

{'score': 0.0,
 'avg_prompt_tokens': 7.0,
 'avg_completion_tokens': 67.0,
 'avg_total_tokens': 74.0,
 'avg_req_latency': 0.005900350709756215,
 'avg_tokens_per_second': 12541.627377783016,
 'avg_cost': 0.0005640000000000002,
 'total_cost': 0.0050760000000000015,
 'success_rate': 1.0,
 'total_time': 0.06302499771118164}